cargo los datos del drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [ ]:
# importamos librerias para el manejo de datos pandas para manejo de datos y numpy operaciones con numeros
import numpy as np
import pandas as pd

#import tools

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
#cargo los datos
train=pd.read_csv('/content/drive/MyDrive/bigdata3/proyecto homicidios/columnas_necesarias.csv')

In [ ]:
train.head()

In [ ]:
train_1 = train[['Fecha', 'Departamento', 'Dia', 'HoraEstandarizada', 'Modalidad', 'Movil_Agresor', 'Movil_Victima', 'Zona', 'Edad', 'Sexo']]
train_1.head()

In [ ]:
#para la columna fecha
#convierto la columna Fecha a tipo fecha con to_datetime()
df = pd.to_datetime(train_1['Fecha'])
#cargo el train_1 en un dataframe para agregar la columna mes
train_1 = pd.DataFrame(train_1)
#agrego la columna mes al train_1
train_1['Mes'] = df.dt.month

#ya se obtubo el mes de la fecha y se agregó esa nueva columna al train_1
#creamos categorias a paetir de los valores existentes para el mes
train_1=pd.concat([train_1,pd.get_dummies(train_1['Mes'],prefix='Mes_')],axis=1)
#miro como van quedando los datos

#hacemos lo mismo para la columna Departamento
train_1=pd.concat([train_1,pd.get_dummies(train_1['Departamento'],prefix='Departamento_')],axis=1)

#cambio texto por número en el dia
train_1['Dia'].replace(
   to_replace= ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'],
    value= [0,1,2,3,4,5,6],
    inplace=True
)
#luego de cambiar el texto por numeros en el dia, normalizo (valor - min) / (max - min)
df_norm = (train_1['Dia'] - train_1['Dia'].min()) / (train_1['Dia'].max()-train_1['Dia'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna dia normalizada al train_1
train_1['dia_normalizado'] = df_norm

#para la columna HoraEstandarizada que se sacó solo la hora mas rápido con excel
train_1=pd.concat([train_1,pd.get_dummies(train_1['HoraEstandarizada'],prefix='Hora_')],axis=1)

#dummies crea categorias a paetir de los valores existentes para Zona
train_1=pd.concat([train_1,pd.get_dummies(train_1['Zona'],prefix='Zona_')],axis=1)

#hacemos lo mismo para la columna Modalidad
train_1=pd.concat([train_1,pd.get_dummies(train_1['Modalidad'],prefix='Modalidad_')],axis=1)

#hacemos lo mismo para la columna Movil_Agresor
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Agresor'],prefix='Movil_Agresor_')],axis=1)

#hacemos lo mismo para la columna Movil_Victima
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Victima'],prefix='Movil_Victima_')],axis=1)

#normalizo la columna edad (valor - min) / (max - min)
df_norm2 = (train_1['Edad'] - train_1['Edad'].min()) / (train_1['Edad'].max()-train_1['Edad'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna edad normalizada al train_1
train_1['edad_normalizada'] = df_norm2

#hacemos lo mismo para la columna sexo
train_1=pd.concat([train_1,pd.get_dummies(train_1['Sexo'],prefix='Sexo_')],axis=1)

train_1.head()

In [ ]:
#borro las columnas que categorice con drop, el implace me dice que ejecuta todo sobre la misma 
# variable, es equivalente a train_1=train_1
train_1.drop(labels=['Fecha'],axis=1,inplace=True)
train_1.drop(labels=['Departamento'],axis=1,inplace=True)
#train_1.drop(labels=['Dia'],axis=1,inplace=True)
train_1.drop(labels=['HoraEstandarizada'],axis=1,inplace=True)
train_1.drop(labels=['Zona'],axis=1,inplace=True)
train_1.drop(labels=['Modalidad'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Agresor'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Victima'],axis=1,inplace=True)
train_1.drop(labels=['Edad'],axis=1,inplace=True)
train_1.drop(labels=['Sexo'],axis=1,inplace=True)
train_1.drop(labels=['Mes'],axis=1,inplace=True)
train_1.head()


In [ ]:
# Y = mX + b  :: Y = wX + b
y = train_1['dia_normalizado']
print(y)

0         0.333333
1         0.666667
2         0.000000
3         0.000000
4         1.000000
            ...   
102300    0.666667
102301    0.666667
102302    0.666667
102303    0.666667
102304    0.666667
Name: dia_normalizado, Length: 102305, dtype: float64


In [ ]:
x = train_1
x.drop('dia_normalizado', axis=1, inplace=True)
print(x)

        Dia  Mes__1  Mes__2  ...  edad_normalizada  Sexo__FEMENINO  Sexo__MASCULINO
0         2       0       0  ...              0.18               0                1
1         4       1       0  ...              0.32               0                1
2         0       0       0  ...              0.48               0                1
3         0       1       0  ...              0.26               0                1
4         6       1       0  ...              0.43               0                1
...     ...     ...     ...  ...               ...             ...              ...
102300    4       0       0  ...              0.13               0                1
102301    4       0       0  ...              0.19               0                1
102302    4       0       0  ...              0.28               1                0
102303    4       0       0  ...              0.34               0                1
102304    4       0       0  ...              0.19               0          

In [ ]:
# Separar los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.176)

In [ ]:
print('== Algunos regresores para prediccion==')
#print('Regresión logística')
KNreg =KNeighborsRegressor()
KNreg.fit(x_train, y_train)
#y_predict = KNreg.predict(x_val)
print('Precisión Regresión lineal (train)', KNreg.score(x_train, y_train))
print('Precisión Regresión lineal (test)', KNreg.score(x_test, y_test))

== Algunos regresores para prediccion==
Precisión Regresión lineal (train) 0.9911080917498654
Precisión Regresión lineal (test) 0.9880348198588924


In [ ]:
y_predictknnre= KNreg.predict(x_val)
import matplotlib.pyplot as plt
plt.plot(y_predictknnre, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("Dia KNeighborsRegressor")
plt.legend()
plt.show()

In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_diaKNreg.pickle", 'wb')
pickle.dump(KNreg,rlfile)
rlfile.close()

In [ ]:
print('== Algunos regresores para prediccion==')
#print('DecisionTreeRegressor')
treeR =DecisionTreeRegressor()
treeR.fit(x_train, y_train)
#y_predict = KNreg.predict(x_val)
print('Precisión Regresión lineal (train)', treeR.score(x_train, y_train))
print('Precisión Regresión lineal (test)', treeR.score(x_test, y_test))

== Algunos regresores para prediccion==
Precisión Regresión lineal (train) 1.0
Precisión Regresión lineal (test) 1.0


In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_diaTreeR.pickle", 'wb')
pickle.dump(treeR,rlfile)
rlfile.close()

In [ ]:
y_predictreeR= treeR.predict(x_val)
import matplotlib.pyplot as plt
plt.plot(y_predictreeR, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("Dia DecisionTreeRegressor")
plt.legend()
plt.show()